# 🎶 **Demo 1: Math Agent with LangChain**
In this notebook we’ll build a simple agent that can add and subtract two numbers using **LangChain** and a LLM.

### 📑 **Table of Contents**

1. 🛠️ Objectives  
2. 📦 Install Dependencies  
3. 🔑 Configure API Key  
4. 👉 Define Addition Tool (`@tool`)  
5. 🔗 Import Tools  
6. 🤖 Create LLM and Initialize Agent
7. 🎯 Interactive Tests  
8. ✅ Conclusions  


### 🛠️ **Objectives**

- Define an **addition**and a **subtraction** tool with `@tool`.  
- Create an **AgentExecutor** to orchestrate both tools.  
- Test the agent with natural prompts.  
---

In [1]:
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables from .env file
load_dotenv(override=True) 

True

In [3]:
# Set the OpenAI API key from the environment variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### 👉 **Define Tools with `@tool`**
We use the `@tool` decorator to register an addition function automatically.

```python
from langchain.agents import tool

@tool
def add_numbers(input_str: str) -> str:
    """
    Takes two numbers separated by a space and returns their sum.

    Args:
        input_str (str): Two numbers in the form "A B", e.g. "3 5"

    Returns:
        str: The sum A + B, or an error message if parsing fails.
    """
    try:
        a, b = map(float, input_str.split())
        return str(a + b)
    except Exception:
        return "error: please send two numbers separated by space, e.g. '3 5'"

```
#### **Key components**

1. **@tool decorator**
   - Registers the function with LangChain
   - Creates tool attributes (.name, .description, .func)
   - Generates JSON schema for validation
   - Transforms regular functions into callable tools

2. **Function name**
   - Used by LLM to select appropriate tool
   - Used as reference in chains and tool mappings
   - Appears in tool call logs for debugging
   - Should clearly indicate the tool's purpose

3. **Type annotations**
   - Enable automatic input validation
   - Create schema for parameters
   - Allow proper serialization of inputs/outputs
   - Help LLM understand required input formats

4. **Docstring**
   - Provides context for the LLM to decide when to use the tool
   - Documents parameter requirements
   - Explains expected outputs and behavior
   - Critical for tool selection by the LLM

5. **Implementation**
   - Executes the actual operation
   - Handles errors appropriately
   - Returns properly formatted results
   - Should be efficient and robust


### 🔗 **Import Tools and AgentExecutor**

In [6]:
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from src.tools.math_tools import add_numbers, subtract_numbers
from src.tools.search_tools import wikipedia_search
from langchain.agents import initialize_agent

In [7]:
print("Tools available:", add_numbers.name, subtract_numbers.name, wikipedia_search.name)

Tools available: add_numbers subtract_numbers wikipedia_search


In [9]:
print("Args:", subtract_numbers.description, 
      add_numbers.description, 
      wikipedia_search.description)

Args: Takes two numbers separated by a space and returns the result of A - B.

Args:
    input_str (str): Two numbers in the form "A B", e.g. "8 3"

Returns:
    str: The difference A − B, or an error message if parsing fails. Takes two numbers separated by a space and returns their sum.

Args:
    input_str (str): Two numbers in the form "A B", e.g. "3 5"

Returns:
    str: The sum A + B, or an error message if parsing fails. Performs a search on Wikipedia and returns a summary of the first result found.

Args:
    query (str): Search term.

Returns:
    str: Summary of the corresponding Wikipedia article.


## 🤖 **Create LLM and Initialize Agent**
Instantiate the LLM (temperature=0 for determinism) and build the agent.

In [10]:
# Configure the LLM
llm = ChatOpenAI(temperature=0, model="gpt-4o")

In [82]:
# Test the LLM with a simple question 
answer = llm.invoke("What is an agent in AI?")
print(answer.content)

In the context of artificial intelligence (AI), an agent is an entity that perceives its environment through sensors and acts upon that environment through actuators. The agent's goal is to perform actions that maximize some notion of cumulative reward or achieve specific objectives. Agents can be simple or complex, and they can operate in a wide range of environments, from highly controlled and predictable to highly dynamic and uncertain.

There are several types of agents in AI, including:

1. **Simple Reflex Agents**: These agents select actions based on the current percept, ignoring the rest of the percept history. They operate on condition-action rules, such as "if condition, then action."

2. **Model-Based Reflex Agents**: These agents maintain an internal model of the world, which helps them handle partially observable environments. They use this model to keep track of the state of the world and make decisions based on both the current percept and the history of percepts.

3. **

#### **Key parameters of `initialize_agent`**

1. **`tools`**

2.  **`llm`**

3. **`agent`**:
   - Specifies the reasoning framework for the agent.
   - `"zero-shot-react-description"` enables:
     - **Zero-shot reasoning**: The agent can solve tasks it hasn't seen before by thinking through the problem step by step.
     - **React framework**: A logical loop of:
       - **Reason** → Think about the task.
       - **Act** → Use a tool to perform an action.
       - **Observe** → Check the tool's output.
       - **Plan** → Decide what to do next.

4. **`verbose`**:
   - If `True`, it prints detailed logs of the agent’s thought process.
   - Useful for debugging or understanding how the agent makes decisions.

In [12]:
# Initialize the agent with the specified tools and LLM 
agent = initialize_agent(
    tools= [add_numbers, subtract_numbers, wikipedia_search],
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

---
### 7️⃣ **Interactive Tests** 🎯

In [13]:
prompts = [
    "Can you add 10 and 15?",
    "Please subtract 7.5 from 20",
    "In 2023, the US GDP was approximately $27.72 trillion, while Canada's was around $2.14 trillion and Mexico's was about $1.79 trillion what is the total.",
    "What is the capital of France?",
    "¿Cuántos habitantes tiene México y Estados Unidos? ¿Cuál es la diferencia entre ellos?",
]

for prompt in prompts:
    print(f"🏷️ Prompt: {prompt}")
    print("➡️ Response:", agent.invoke({"input": prompt}), "\n")

🏷️ Prompt: Can you add 10 and 15?


> Entering new AgentExecutor chain...
Question: Can you add 10 and 15?
Thought: I need to use the add_numbers tool to find the sum of 10 and 15.
Action:
```
{
  "action": "add_numbers",
  "action_input": "10 15"
}
```

Observation: 25.0
Thought:I now know the final answer. 

Final Answer: The sum of 10 and 15 is 25.

> Finished chain.
➡️ Response: {'input': 'Can you add 10 and 15?', 'output': 'The sum of 10 and 15 is 25.'} 

🏷️ Prompt: Please subtract 7.5 from 20


> Entering new AgentExecutor chain...
Question: Please subtract 7.5 from 20
Thought: I need to perform a subtraction operation with the numbers 20 and 7.5.
Action:
```
{
  "action": "subtract_numbers",
  "action_input": "20 7.5"
}
```

Observation: 12.5
Thought:I now know the final answer. 

Final Answer: 12.5

> Finished chain.
➡️ Response: {'input': 'Please subtract 7.5 from 20', 'output': '12.5'} 

🏷️ Prompt: In 2023, the US GDP was approximately $27.72 trillion, while Canada's was arou

In [15]:
agent.invoke({"input": "¿Cuál es el plato típico de Antioquía y cuánto costaría invitar a 5 personas a almorzar?"})



> Entering new AgentExecutor chain...
Question: ¿Cuál es el plato típico de Antioquía y cuánto costaría invitar a 5 personas a almorzar?
Thought: Primero, buscaré información sobre el plato típico de Antioquía.
Action:
```
{
  "action": "wikipedia_search",
  "action_input": "plato típico de Antioquía"
}
```

Observation: No Wikipedia page was found for this term.
Thought:Thought: Intentaré buscar información sobre el plato típico de Antioquía utilizando un término más general.
Action:
```
{
  "action": "wikipedia_search",
  "action_input": "Bandeja Paisa"
}
```

Observation: Bandeja paisa (Spanish pronunciation: [banˌdexa ˈpai̯sa]; lit. 'Paisa platter'), with variations known as bandeja de arriero, bandeja montañera, and bandeja antioqueña, is one of the most representative meals in Colombian cuisine, especially of the Antioquia department and the Paisa region, as well as with the Colombian Coffee-Growers Axis (the departments of Caldas, Quindío and Risaralda, Tolima and Valle del Ca

{'input': '¿Cuál es el plato típico de Antioquía y cuánto costaría invitar a 5 personas a almorzar?',
 'output': 'El plato típico de Antioquía es la Bandeja Paisa. Invitar a 5 personas a almorzar costaría aproximadamente 150,000 pesos colombianos, considerando un costo promedio de 30,000 pesos por persona.'}

### ✅ **Conclusions**

- We defined **add_numbers** and **subtract_numbers** tools using `@tool`.  
- The agent chooses and invokes the correct tool based on your prompt.  
- In just a few lines, we have a full “prompt → reasoning → tool call → response” flow.  

**Ready for the next orchestration demo!**